In [23]:
from reaktoro import *
from reaktplot import *
from numpy import arange

In [24]:
db = PhreeqcDatabase("phreeqc.dat")

gaseousphase = GaseousPhase("CO2(g)")
gaseousphase.setActivityModel(ActivityModelPengRobinson())

system = ChemicalSystem(db, gaseousphase)

In [25]:
temperatures = [25, 50, 100, 150]  # temperatures in °C
pressures = arange(1.0, 500.0, 5.0)  # pressures in atm

state = ChemicalState(system)
state.set("CO2(g)", 1.0, "mol")

molarvolumes = { "25°C": [], "50°C": [], "100°C": [], "150°C": [] }

m3_to_liter = 1000.0

for Tc in temperatures:
    for Pa in pressures:
        state.temperature(Tc, "celsius")
        state.pressure(Pa, "atm")

        state.props().update(state)

        V = state.props().phaseProps("GaseousPhase").molarVolume() * m3_to_liter

        molarvolumes[f"{Tc}°C"].append(V.val())

In [26]:
# Experimental data presented in:  
#   * Michels et al. (1935) Proc. Royal Soc. 153, 201, 214; 
#   * Michels et al. (1937) Proc. Royal Soc. 160, 358
# and collected from https://www.hydrochemistry.eu/exmpls/CO2_VP.phr
# Units for molar volume (V) and pressure (P) are liter/mol and atm respectively.
expdata = {
    "V(25°C)":  [ 1.195, 0.955, 0.796, 0.684, 0.596, 0.528, 0.476, 0.432, 0.490, 0.324, 0.246, 0.200, 0.058, 0.053, 0.049, 0.046, 0.043, 0.040, 0.038, 0.178, 0.114, 0.068, 0.062 ],
    "P(25°C)":  [ 18.537, 22.572, 26.371, 29.865, 33.307, 36.494, 39.407, 42.170, 38.585, 50.489, 57.871, 62.155, 74.739, 109.870, 180.783, 308.620, 523.452, 864.844, 1325.170, 63.450, 63.462, 63.459, 63.550 ],
    "V(50°C)":  [ 0.476, 0.324, 0.246, 0.200, 0.168, 0.146, 0.132, 0.119, 0.109, 0.109, 0.099, 0.091, 0.084, 0.078, 0.073, 0.068, 0.064, 0.064, 0.058, 0.053, 0.049, 0.046, 0.043 ],
    "P(50°C)":  [ 44.982, 59.524, 70.782, 79.158, 85.738, 90.682, 94.048, 97.232, 100.111, 100.118, 103.257, 106.665, 110.491, 115.297, 121.776, 130.848, 142.376, 145.229, 180.898, 245.038, 350.279, 518.686, 781.537 ],
    "V(100°C)": [ 0.476, 0.490, 0.324, 0.246, 0.200, 0.168, 0.146, 0.132, 0.119, 0.109, 0.099, 0.091, 0.084, 0.078, 0.073, 0.068, 0.064, 0.064, 0.058, 0.053, 0.049, 0.043, 0.040 ],
    "P(100°C)": [ 55.798, 54.413, 76.745, 95.130, 110.947, 125.564, 138.669, 149.054, 160.256, 171.567, 184.991, 199.951, 216.386, 235.395, 258.216, 286.180, 317.264, 324.641, 403.553, 522.315, 691.824, 1289.390, 1792.290 ],
    "V(150°C)": [ 0.476, 0.490, 0.324, 0.246, 0.200, 0.168, 0.146, 0.132, 0.119, 0.109, 0.109, 0.099, 0.091, 0.084, 0.078, 0.073, 0.068, 0.064, 0.064, 0.058, 0.053, 0.049, 0.043 ],
    "P(150°C)": [ 66.370, 64.628, 93.426, 111.587, 141.534, 164.001, 185.206, 202.746, 222.250, 242.450, 242.478, 266.859, 294.204, 324.034, 357.992, 397.723, 444.846, 495.431, 507.212, 628.422, 799.215, 1030.05, 1782.41 ],
}

In [30]:
fig = Figure()
fig.title("PENG-ROBINSON EOS FOR CO<sub>2</sub>")
fig.xaxisTitle("Molar Volume [liter/mol]")
fig.yaxisTitle("Pressure [atm]")
fig.xaxisRange(0.0, 0.50)
fig.yaxisRange(1.0, 500.0)
for Tc in temperatures:
    fig.drawLine(molarvolumes[f"{Tc}°C"], pressures, f"{Tc}°C (Reaktoro: Peng-Robinson)")
    fig.drawMarkers(expdata[f"V({Tc}°C)"], expdata[f"P({Tc}°C)"], f"{Tc}°C (Michels et al. 1935,1937)")

fig.show()